In [1]:
import geopandas as gpd
import geopy
import shapely as sh
from shapely.geometry import Point

In [2]:
def coordToArray(geopdcoord):
    coord = str(geopdcoord['geometry'][0])
    j = coord.replace("POINT (","") #remove palavra POINT (
    i = j.replace(")","").split(" ") # remove ultimo parenteses e splita o que sobrou resultando num array com as 2 coord
    ponto = Point(float(i[0]),float(i[1]))
    return ponto

In [21]:
coor = gpd.tools.geocode("Mario Tavares De Souza 280, Porto Alegre", timeout=5, provider='here', app_id='dBJNCvIN6lV7lK7BmDbV', app_code='XxYqC9TrTRW0DPUBZrjeaQ')
#rua = gpd.tools.reverse_geocode([Point(-51.12429,-30.09012)], timeout=5,provider='here', app_id='dBJNCvIN6lV7lK7BmDbV', app_code='XxYqC9TrTRW0DPUBZrjeaQ')
#print(coor.geometry)
#rua

In [29]:
latlong = coordToArray(coor)
latlong

In [66]:
coordendas['address'][0]

'Rua Mário Tavares de Souza, 280, Agronomia, Porto Alegre - RS, 91540-150, Brazil, Brazil'

In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_excel("RouboRecupVec2.xlsx")
#df.head()

In [5]:
idhere='dBJNCvIN6lV7lK7BmDbV'
codehere='XxYqC9TrTRW0DPUBZrjeaQ'

In [124]:
#seleciona apenas linhas que tem nome de rua diferente de vazio ou -  e de um ano específico
mask = (pd.isna(df['Geocode']) & (pd.notna(df['Nome Rua'])|(df['Nome Rua']!='-')) & ((df['Ano']>=2006) &(df['Ano']<=2009)))#máscara para selecionar apenas dados sem geo code e com rua 
dfSgeo = df.loc[mask]

In [125]:
dfSgeo.shape

(138243, 18)

In [58]:
#dfSgeo.dtypes

In [126]:
mask = (((dfSgeo['Município']=='Porto Alegre')|
         (dfSgeo['Município']=='Canoas')|
         (dfSgeo['Município']=='Viamão')| 
         (dfSgeo['Município']=='Cachoeirinha')|
         (dfSgeo['Município']=='Gravataí')|
         (dfSgeo['Município']=='São Leopoldo')|
         (dfSgeo['Município']=='Novo Hamburgo')|
         (dfSgeo['Município']=='Alvorada')|
         (dfSgeo['Município']=='Sapucaia do Sul')|
         (dfSgeo['Município']=='Guaíba')|
         (dfSgeo['Município']=='Eldorado do Sul')|
         (dfSgeo['Município']=='Esteio')) & pd.notna(dfSgeo['Nro']))

dfteste = dfSgeo.loc[mask]
dfteste.shape

(69860, 18)

In [105]:
newgeocode = []

In [106]:
#LAÇO DE REPETIÇÃO QUE PEGA COLUNAS DE RUA, NRO E CIDADE E CONSULTA NO HERE WEGO MAPS
import time
i=0
for i in range(len(dfteste.index)):
#for i in range(5):    
    time.sleep(0.4) #tempo pro programa pensar
    endereco = str(dfteste.iat[i,14])+' '+str(dfteste.iat[i,15])+', '+str(dfteste.iat[i,12]) #colunas de rua, nro e cidade
    #print(endereco)
    
    tentativas = [3,5,7,10,15]    
    for tmout in tentativas: #laço para tentar pesquisar com timouts diferentes
        try:
            coordenadas=gpd.tools.geocode(endereco, timeout=tmout,provider='here', app_id=idhere, app_code=codehere)
            newgeocode.append(coordToArray(coordenadas))
            #print(tmout)
            break # se der certo ele quebra o for
        except:
            if tmout == 15: #se chegou até o time out 15 e entrou na exceção então insere coordenada do guaíba
                coordenadas = Point(-51.278229,-30.068491) #coordenadas no meio do lago Guaíba
                newgeocode.append(coordenadas)
                break
            else:
                continue #se ainda não chegou no tentou o time out 15 então continua tentando      

In [107]:
from datetime import datetime
# datetime object containing current date and time
now = datetime.now()
 
print("now =", now)

now = 2019-11-01 13:21:48.520408


In [108]:
len(newgeocode)

11235

In [109]:
dfNgeo = pd.DataFrame({'Geocode Novo':newgeocode})
dfteste.index = range(len(dfteste)) #reseta os index do conjunto que foi pesquisado para possibilitar o concat
dfFinal = pd.concat([dfteste,dfNgeo], axis=1, sort=False) #une o Df de pesquisa com o df de coordendas obtidas
dfFinal.head()
#dfFinal['New Geocode'] = newgeocode

,Órgão Registro,Ano Registro,Número Inteiro Ocorrência,Ano,Hora Minuto,Bairro Fato,Geocode,Placa,Marca,Tipo de Veículo,Ano Fabricação,Ano Modelo,Município,Data Fato,Nome Rua,Nro,Tipo Fato,Fato,Geocode Novo
0,100310,2005,8224,2005,21:00:00,NaN,NaN,LYG8322,FIAT/TEMPRA SX,AUTOMOVEL,1997,1997,Porto Alegre,2005-07-27,JOAO TELLES,195,Consumado,ROUBO DE VEICULO,POINT (-51.2123532 -30.0310547)
1,100305,2005,5397,2005,22:00:00,NaN,NaN,MBW2977,FORD FOCUS2.0L,AUTOMOVEL,2001,2001,Porto Alegre,2005-08-30,LUDOLFO BOEHL,388,Consumado,ROUBO DE VEICULO,POINT (-51.2085 -30.08215)
2,100317,2005,212,2005,13:00:00,NaN,NaN,IHE0192,M.BENZ/L 608 D,CAMINHAO,1981,1981,Porto Alegre,2005-01-09,SETE DE ABRIL,55,Consumado,RECUPERACAO DE VEICULO,POINT (-51.20969 -30.01713)
3,100313,2005,8381,2005,09:00:00,NaN,NaN,IAR1811,VW/SANTANA CG,AUTOMOVEL,1986,1986,Porto Alegre,2005-10-04,FREDERICO CARLOS GOMES,309,Consumado,RECUPERACAO DE VEICULO,POINT (-51.20393 -30.11924)
4,100450,2005,4652,2005,18:00:00,NaN,NaN,ILZ7970,VW/GOL 1.0,AUTOMOVEL,2004,2005,Guaíba,2005-07-23,GETULIO VARGAS,1495,Consumado,ROUBO DE VEICULO,POINT (-52.2400793 -27.8893899)


In [110]:
dfFinal.head(50)

,Órgão Registro,Ano Registro,Número Inteiro Ocorrência,Ano,Hora Minuto,Bairro Fato,Geocode,Placa,Marca,Tipo de Veículo,Ano Fabricação,Ano Modelo,Município,Data Fato,Nome Rua,Nro,Tipo Fato,Fato,Geocode Novo
0,100310,2005,8224,2005,21:00:00,NaN,NaN,LYG8322,FIAT/TEMPRA SX,AUTOMOVEL,1997,1997,Porto Alegre,2005-07-27 00:00:00,JOAO TELLES,195,Consumado,ROUBO DE VEICULO,POINT (-51.2123532 -30.0310547)
1,100305,2005,5397,2005,22:00:00,NaN,NaN,MBW2977,FORD FOCUS2.0L,AUTOMOVEL,2001,2001,Porto Alegre,2005-08-30 00:00:00,LUDOLFO BOEHL,388,Consumado,ROUBO DE VEICULO,POINT (-51.2085 -30.08215)
2,100317,2005,212,2005,13:00:00,NaN,NaN,IHE0192,M.BENZ/L 608 D,CAMINHAO,1981,1981,Porto Alegre,2005-01-09 00:00:00,SETE DE ABRIL,55,Consumado,RECUPERACAO DE VEICULO,POINT (-51.20969 -30.01713)
3,100313,2005,8381,2005,09:00:00,NaN,NaN,IAR1811,VW/SANTANA CG,AUTOMOVEL,1986,1986,Porto Alegre,2005-10-04 00:00:00,FREDERICO CARLOS GOMES,309,Consumado,RECUPERACAO DE VEICULO,POINT (-51.20393 -30.11924)
4,100450,2005,4652,2005,18:00:00,NaN,NaN,ILZ7970,VW/GOL 1.0,AUTOMOVEL,2004,2005,Guaíba,2005-07-23 00:00:00,GETULIO VARGAS,1495,Consumado,ROUBO DE VEICULO,POINT (-52.2400793 -27.8893899)
5,100315,2005,4270,2005,21:00:00,NaN,NaN,ICK5940,GM/CORSA WIND,AUTOMOVEL,1994,1994,Porto Alegre,2005-06-10 00:00:00,ALBION,258,Consumado,ROUBO DE VEICULO,POINT (-51.1658376 -30.0599266)
6,100311,2005,1934,2005,20:00:00,NaN,NaN,IDA2531,GM/MONZA SL/E EFI,AUTOMOVEL,1991,1992,Porto Alegre,2005-03-08 00:00:00,ERNESTO DI PRIMIO BECK,355,Consumado,RECUPERACAO DE VEICULO,POINT (-51.17494 -30.07212)
7,100312,2005,2511,2005,20:00:00,NaN,NaN,IMD9947,GM/CORSA WIND,AUTOMOVEL,2004,2005,Porto Alegre,2005-04-06 00:00:00,PASSOS FIGUEIROA,756,Consumado,RECUPERACAO DE VEICULO,POINT (-51.13253 -29.99164)
8,100424,2005,2521,2005,21:00:00,NaN,NaN,IKG0089,HONDA/CG 125 TITAN ES,MOTOCICLETA,2001,2002,Alvorada,2005-04-06 00:00:00,BANDEIRANTES,200,Consumado,ROUBO DE VEICULO,POINT (-51.0778683 -30.0034019)
9,100320,2005,5411,2005,22:00:00,NaN,NaN,AMA7641,VW/PARATI 1.8,CAMINHONETE,2005,2005,Porto Alegre,2005-08-29 00:00:00,JACUI,859,Consumado,RECUPERACAO DE VEICULO,POINT (-51.23881 -30.07872)


In [111]:
#Cria tabela
#with pd.ExcelWriter('Georreferenciado2005.xlsx') as writer:
#    dfFinal.to_excel(writer, sheet_name='Teste')    

In [115]:
#Adiciona mais uma aba a um arquivo existente
with pd.ExcelWriter('Georreferenciado2005.xlsx', mode='a', engine='openpyxl') as writer: 
    dfFinal.to_excel(writer, sheet_name='2006') 

In [11]:
import folium
from folium import plugins

In [112]:
canoas = folium.Map(location=[-29.923361,-51.177189],zoom_start=10)
conjunto=[]
for coordenadas in newgeocode:
    conjunto.append([coordenadas.y,coordenadas.x])
    
canoas.add_child(plugins.HeatMap(conjunto))
canoas
    #folium.Marker(location=[coordenadas.x,coordenadas.y]).add_to(canoas)
#folium.Marker(location=[newgeocode[0].y,newgeocode[0].x]).add_to(canoas)

In [127]:
69860/50/60

23.28666666666667